## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `rerank_rag.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [3]:
# LISTANDO AS PERGUNTAS A SEREM RESPONDIDAS
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [4]:
# Carregando os modelos OpenAI - Embedding e Chat
# O modelo de embedding será usado para gerar representações vetoriais dos textos
# O modelo de chat será usado para gerar respostas a partir do contexto fornecido

# Carregando o modelo de embeddings da OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # Modelo de embedding para gerar vetores

# Inicializando o modelo de linguagem ChatGPT (gpt-3.5-turbo)
# Este modelo será usado para gerar respostas para as perguntas com base no contexto
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # Nome do modelo de linguagem
    max_tokens=500,              # Número máximo de tokens na resposta gerada
)


In [5]:
# Carregando o PDF
# O carregador (loader) será responsável por carregar o PDF e dividir o conteúdo em páginas

pdf_link = "os-sertoes.pdf"  # Caminho para o arquivo PDF

# Inicializando o carregador de PDF
# O parâmetro 'extract_images=False' garante que imagens não sejam extraídas do PDF
loader = PyPDFLoader(pdf_link, extract_images=False)

# Carregando o conteúdo do PDF e dividindo-o em páginas
# A função 'load_and_split' divide o PDF em uma lista de páginas para processamento posterior
pages = loader.load_and_split()


In [6]:
# Separando o conteúdo em chunks (pedaços menores)
# O objetivo é dividir o conteúdo do PDF em pedaços menores para processamento posterior

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          # Tamanho máximo de cada pedaço (chunk)
    chunk_overlap=20,         # Sobreposição entre os pedaços (chunks)
    length_function=len,      # Função para calcular o comprimento dos pedaços
    add_start_index=True      # Adiciona o índice de início a cada pedaço
)

# Dividindo o conteúdo do PDF (páginas) em pedaços menores (chunks)
chunks = text_splitter.split_documents(pages)


In [7]:
# Salvando os chunks no VectorDB
# O banco de dados vetorial (VectorDB) armazena os pedaços (chunks) em formato vetorial para recuperação posterior

vectordb = Chroma(embedding_function=embeddings)  # Criando o banco de dados vetorial utilizando o modelo de embeddings

# Carregando os documentos (chunks) no VectorDB
# Aqui, estamos armazenando os pedaços (chunks) no banco de dados vetorial para que possam ser recuperados mais tarde
vectordb.add_documents(chunks)

# Carregando o DB e configurando o recuperador (retriever)
# O recuperador irá buscar os 10 documentos mais relevantes baseados na consulta
naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})


/tmp/ipykernel_22507/3538103667.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(embedding_function=embeddings)  # Criando o banco de dados vetorial utilizando o modelo de embeddings


In [8]:
# Configurando o Reranker para melhorar a qualidade da recuperação
# O reranker é responsável por reclassificar os documentos retornados pelo recuperador, mantendo os mais relevantes

rerank = CohereRerank(model="rerank-v3.5", top_n=3)  # Rerank com o modelo "rerank-v3.5", retornando os 3 melhores resultados

# Criando o ContextualCompressionRetriever
# O compressor contextual é responsável por combinar o reranker com o recuperador base para melhorar a precisão das respostas
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,       # Usando o reranker para melhorar a recuperação
    base_retriever=naive_retriever,  # Usando o recuperador base (naive_retriever)
)


In [9]:
# Definindo o template para o prompt de Chat
# O template será usado para estruturar a pergunta e o contexto que serão passados para o modelo de linguagem
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

# Criando o prompt do chat a partir do template definido
# 'ChatPromptTemplate.from_template' cria um prompt com base no template fornecido
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [10]:
# Configurando o processo de recuperação paralelo
# A recuperação paralelo permite passar a pergunta e o contexto para o recuperador de forma independente

setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})  # Processamento paralelo para pergunta e contexto

# Inicializando o parser para a saída
# O parser converte a resposta gerada pelo modelo de linguagem para um formato desejado
output_parser = StrOutputParser()

# Criando a cadeia de recuperação com compressão
# O fluxo de execução inclui o setup de recuperação, o prompt, o modelo de linguagem e o parser de saída
compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser


In [11]:
# Função para responder a uma pergunta utilizando o fluxo de recuperação com compressão
# A função usa a cadeia de recuperação configurada para processar a pergunta e gerar uma resposta

def answer_question(question: str):
    # Chamando a cadeia de recuperação para processar a pergunta e retornar a resposta
    return compressor_retrieval_chain.invoke(question)


In [12]:
# Iterando sobre as perguntas e obtendo as respostas
# 'enumerate' permite obter tanto o índice quanto a pergunta de cada item na lista
for index, question in enumerate(questions):
    # Obtendo a resposta para a pergunta atual utilizando a função 'answer_question'
    resposta = answer_question(question)
    # Exibindo o número da pergunta, a pergunta e a resposta gerada
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como um local de extremos, onde há uma alternância entre períodos de fertilidade e de seca intensa. Ele destaca a importância da água como recurso escasso e fundamental para a sobrevivência dos habitantes da região. O autor ressalta que a natureza cria um jogo de antíteses nesse ambiente, com áreas férteis contrastando com estepes áridas. Ele também menciona a influência da ação humana, como o desmatamento desordenado, na degradação do solo e na ocorrência de queimadas, que contribuem para a intensificação da seca e para o sofrimento da população local. Euclides da Cunha mostra como a relação entre o homem e o meio ambiente no sertão nordestino é marcada por desafios e dificuldades decorrentes das características naturais da região.'}
{'numero': 1, 'perg